In [ ]:
# !pip install scipy
# !pip install pysindy plotly

In [ ]:
import numpy as np
import plotly.graph_objs as go
from scipy.integrate import solve_ivp
import pysindy as ps

# Параметры системы Лоренца
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0

# Уравнения Лоренца
def lorenz(t, state):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Начальные условия и время интегрирования
initial_state = [1.0, 1.0, 1.0]
t_span = (0, 49)  # временной интервал
t_eval = np.linspace(t_span[0], t_span[1], 10000)  # временные точки

# Интеграция системы
sol = solve_ivp(lorenz, t_span, initial_state, t_eval=t_eval)
x = sol.y[0]

# Создание векторов задержки
tau = 13
embedding_dim = 3

def delay_embedding(data, delay, dim):
    n = len(data)
    embedded_data = np.zeros((n - (dim - 1) * delay, dim))
    for i in range(dim):
        embedded_data[:, i] = data[i * delay:n - (dim - 1 - i) * delay]
    return embedded_data

# def delay_embedding(data, delay, dim):
#     n = len(data)
#     embedded_data = np.zeros((n - (dim - 1) * delay, dim))
#     embedded_data[:, 0] = data[0:n - (dim - 1) * delay]
#     embedded_data[:, 1] = data[delay:n - (dim - 1 - 1) * delay]
#     embedded_data[:, 2] = data[2 * delay:n - (dim - 1 - 2) * delay]
#     return embedded_data

embedded_data = delay_embedding(x, tau, embedding_dim)

# Задержка времени (шаг)
dt = t_eval[1] - t_eval[0]

# Обучение модели SINDy
model = ps.SINDy()
model.fit(embedded_data, t=dt)
model.print()

# Прогнозирование и визуализация
simulated_data = model.simulate(embedded_data[0], t_eval)

# Построение интерактивного графика с Plotly
fig = go.Figure()

# Восстановленные данные
fig.add_trace(go.Scatter3d(
    x=embedded_data[:, 0], y=embedded_data[:, 1], z=embedded_data[:, 2],
    mode='lines', name='Reconstructed',
    line=dict(color='blue', width=2)
))

# Симулированные данные
fig.add_trace(go.Scatter3d(
    x=simulated_data[:, 0], y=simulated_data[:, 1], z=simulated_data[:, 2],
    mode='lines', name='Simulated',
    line=dict(color='red', width=2)
))

# Настройки графика
fig.update_layout(
    title='Reconstructed Lorenz Attractor and SINDy Simulation',
    scene=dict(
        xaxis_title='x(t)',
        yaxis_title=f'x(t-{tau})',
        zaxis_title=f'x(t-{2*tau})'
    ),
    width=800,
    height=600
)

fig.show()